In [1]:
#imports
import numpy as np, pandas as pd, os, openpyxl, re

In [2]:
with open("unispd.txt","r") as f:
    txt = list(filter(lambda x:not re.match(r"\s*$",x), f.read().split("\n-------end session-------\n")))

Runtime was capped at 2000ms and 5 trials of every setting done and averaged out.

In [3]:
#initializing each necessary dataframe
df1k = pd.DataFrame(columns=['nthreads','Serial','LockFree','TASLock','pthread_mutex','ALock','CLHLock'])
df1k.loc[:,'nthreads'] = [2, 3, 4, 8, 14, 28]
df1k.set_index('nthreads',inplace=True)
df1k.loc[:,:] = np.zeros((6,6))

df2k, df4k, df8k = [df1k.copy() for _ in range(3)]

In [4]:
typedict = {-2:'Serial', -1:'LockFree', 0:'TASLock', 1:'pthread_mutex', 2:'ALock', 3:'CLHLock'}
workdict = {1000:df1k,2000:df2k,4000:df4k,8000:df8k}

In [5]:
#parse
for s in range(len(txt)):
    W = int(re.search(r'(?<=pkwk: )[0-9]+',txt[s]).group())
    nthreads = int(re.search(r'(?<=nsrc: )[0-9]+',txt[s]).group()) + 1
    processed = int(re.search(r"(?<=processed: )[0-9]+",txt[s]).group())
    strat = int(re.search(r'(?<=opmode: )\-*[0-9]+',txt[s]).group())
    locktype = int(re.search(r'(?<=lockmode: )\-*[0-9]+',txt[s]).group())
    if strat==-1 and locktype==-1:
        locktype = -2
    workdict[W].loc[nthreads,typedict[locktype]] += processed

In [6]:
#turn into avg throughput ratio to serial, write to excel
with pd.ExcelWriter('unispdEX.xlsx') as writer:
    for key in workdict.keys():
        workdict[key] /= 5*2000
        for j in range(1,6):
            workdict[key].iloc[:,j] /= workdict[key].iloc[:,0]
        workdict[key].to_excel(writer,sheet_name="W={}".format(key))

In [7]:
df1k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,390.3327,0.924721,0.916703,0.909589,0.919666,0.922645
3,401.8437,1.824588,1.809401,1.784259,1.800646,1.789421
4,410.2800,2.652668,2.638558,2.543091,2.615185,2.573929
8,410.2719,6.146464,6.024283,5.786854,6.042499,5.949568
14,409.9971,7.150103,7.261214,6.838095,7.021358,7.211014
28,409.9904,0.016628,0.017726,0.018682,0.017703,0.017346


In [8]:
df2k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,208.5970,0.934206,0.930674,0.927210,0.929718,0.933996
3,208.4249,1.875263,1.865229,1.850684,1.859808,1.847951
4,208.1612,2.784193,2.775221,2.722182,2.752418,2.740021
8,208.6868,6.355712,6.294889,6.235243,6.270723,6.281652
14,208.7450,11.365621,11.246899,11.014508,10.023041,10.986171
28,208.8188,0.035832,0.035731,0.034549,0.033160,0.035652


In [9]:
df4k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,104.8873,0.966262,0.964524,0.963702,0.964274,0.965076
3,105.0109,1.916691,1.912912,1.905035,1.909007,1.903229
4,105.1216,2.848880,2.841222,2.818275,2.834068,2.825948
8,104.9835,6.521356,6.512883,6.383636,6.448485,6.469741
14,105.1338,11.517593,11.348165,11.415127,10.049050,11.225086
28,105.1340,0.070063,0.068956,0.069546,0.066195,0.069453


In [10]:
df8k

,Serial,LockFree,TASLock,pthread_mutex,ALock,CLHLock
nthreads,,,,,,
2,52.6994,0.981871,0.980068,0.979641,0.980856,0.981078
3,52.8352,1.937907,1.936620,1.932808,1.933684,1.927777
4,52.7584,2.878967,2.874996,2.864621,2.872782,2.868192
8,52.7361,6.590224,6.579210,6.502747,6.503096,6.532489
14,52.8825,11.495759,11.190099,11.247904,11.490179,11.477614
28,52.9210,0.131239,0.143786,0.126447,0.126721,0.137989
